![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

## Load Two Months of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table if it exists to start fresh.

In [ ]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc.taxis;

## First create the table

In [ ]:
%%sql

DROP TABLE IF EXISTS nyc.taxis;

In [ ]:
%%sql

CREATE TABLE nyc.taxis (
    VendorID              bigint,
    tpep_pickup_datetime  timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count       double,
    trip_distance         double,
    RatecodeID            double,
    store_and_fwd_flag    string,
    PULocationID          bigint,
    DOLocationID          bigint,
    payment_type          bigint,
    fare_amount           double,
    extra                 double,
    mta_tax               double,
    tip_amount            double,
    tolls_amount          double,
    improvement_surcharge double,
    total_amount          double,
    congestion_surcharge  double,
    airport_fee           double
)
USING iceberg
PARTITIONED BY (days(tpep_pickup_datetime))

# Write a month of data

In [ ]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2022-01.parquet")
df.writeTo("nyc.taxis").append()

In [ ]:
%%sql

SELECT *
FROM nyc.taxis

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.snapshots

# Write a month of data

In [ ]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2022-02.parquet")
df.writeTo("nyc.taxis").append()

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.snapshots
ORDER BY committed_at DESC

## Manifest lists

Now we'll list all the manifests. This is the abovemention `manifest_list` of the current snapshot.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.manifests

# Manifests

The next layer is the manifests that has references to the Parquet files.

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.files

# Flexibility of partitioning

We can easily change the partitioning of the table

In [ ]:
%%sql

SELECT * FROM nyc.taxis.partitions

In [ ]:
%%sql

ALTER TABLE nyc.taxis DROP PARTITION FIELD days(tpep_pickup_datetime)

In [ ]:
%%sql

ALTER TABLE nyc.taxis ADD PARTITION FIELD hours(tpep_pickup_datetime)

In [ ]:
%%sql

SELECT * FROM nyc.taxis.partitions

In [ ]:
%%sql

CALL system.rewrite_data_files('nyc.taxis')

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.files

In [ ]:
%%sql

SELECT *
FROM nyc.taxis.snapshots
ORDER BY committed_at DESC